In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

from PIL import Image
# from torchinfo import summary
import torch
import os
import warnings
warnings.filterwarnings("ignore")
from typing import Tuple

from PIL import Image
import torch
import torch.nn as nn
# import torch.nn.functional as F
import torchvision.transforms as T
from torchvision.transforms import functional as F
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

from datasets import load_dataset
import pandas as pd

import random

import wandb
# wandb.login("1fa58b4e42c64c2531b3abeb43c04f5991be307e")

torch.cuda.empty_cache()


In [2]:
# !wandb login 1fa58b4e42c64c2531b3abeb43c04f5991be307e

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [2]:
print('Number CUDA Devices:', torch.cuda.device_count())
print ('Current cuda device: ', torch.cuda.current_device(), ' **May not correspond to nvidia-smi ID above, check visibility parameter')

Number CUDA Devices: 1
Current cuda device:  0  **May not correspond to nvidia-smi ID above, check visibility parameter


In [7]:
from sklearn.model_selection import train_test_split

labelsTable = pd.read_csv('../mnt/local/data/kalexu97/trainLabels.csv')
root_dir = '../mnt/local/data/kalexu97/train'
labelsTable['image_path'] = labelsTable['image'].apply(lambda x: os.path.join(root_dir, x+'.jpeg'))
labelsTable['label'] = labelsTable['level']

labelsTable['pr_image_path'] = labelsTable['image'].apply(
    lambda x: os.path.join('../mnt/local/data/kalexu97/preprocessed_train/images', x+'.jpeg'))
labelsTable['mask_path'] = labelsTable['image'].apply(
    lambda x: os.path.join('../mnt/local/data/kalexu97/preprocessed_train/masks', x+'.jpeg'))

labelsTable = labelsTable.drop(columns=['image', 'level'], axis=1)

train_dataset, test_dataset = train_test_split(labelsTable, test_size=0.2)

In [8]:

# oversampling just repeating minority class items 
# enought times to be equal to major dataset in size

##############################################################################################

# max_size = train_dataset['label'].value_counts().max()
# lst = [train_dataset]
def resample(_dataset, ratio = 3):
    min_size = _dataset['label'].value_counts().min()
    lst = []
    added_unique_rows = 0
    all_n_rows = 0
    
    for class_index, group in _dataset.groupby('label'):
        # lst.append(group.sample(max_size-len(group), replace=True))
        all_n_rows += len(group)
        if class_index == 0:
            added_unique_rows += min_size*ratio
            lst.append(group.sample(min_size*ratio, replace=False))
        else:
            if len(group) > min_size*ratio:
                added_unique_rows += min_size*ratio
                lst.append(group.sample(min_size*ratio, replace=False))
            else:
                lst.append(group)
                added_unique_rows += len(group)
                lst.append(group.sample(min_size*ratio-len(group), replace=True))
        
    _dataset = pd.concat(lst)

    for class_index, group in _dataset.groupby('label'):
        print(f'{class_index}: length: {len(group)}')

    print('N_added_rows: ', added_unique_rows)
    print('N_all_rows: ', all_n_rows)
    print('Ratio of used rows: ', added_unique_rows/all_n_rows)
    return _dataset

##############################################################################################
train_dataset = resample(train_dataset, ratio = 20)
# test_dataset = resample(test_dataset, ratio = 1)


0: length: 11520
1: length: 11520
2: length: 11520
3: length: 11520
4: length: 11520
N_added_rows:  18991
N_all_rows:  28100
Ratio of used rows:  0.6758362989323844


In [9]:
mean, std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
from datasets import Dataset
from transformers import ViTImageProcessor
from transformers import AutoImageProcessor

# model_name_or_path = 'google/vit-base-patch16-224-in21k'
# model_name_or_path = "microsoft/swinv2-tiny-patch4-window8-256"
model_name_or_path = "microsoft/swin-base-patch4-window12-384"
 
# processor = ViTImageProcessor.from_pretrained(model_name_or_path)
image_processor = AutoImageProcessor.from_pretrained(model_name_or_path)

normalize = T.Normalize(mean=image_processor.image_mean, std=image_processor.image_std)

size_model = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)

#####################################################################################
# size = size_model + 32 # 256
# re_size = 224
re_size = size_model[0]

def get_normalize():

    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    normalize = T.Normalize(mean=mean, std=std)
    
    return normalize
    
normalize = get_normalize()

tra_train = T.Compose([
    T.RandomResizedCrop(re_size, scale=(0.7, 1.0)),
    T.RandomHorizontalFlip(),
    T.ColorJitter(0.3, 0.3, 0.3, 0.3),
    T.RandomGrayscale(),
    T.ToTensor(), #0.5
    normalize,
])

tra_test = T.Compose([
    T.Resize((re_size, re_size)),
    T.ToTensor(),
    normalize
])

tra_mask= T.Compose([
            T.Resize(re_size),
            T.ToTensor()])

#######################################################################

# size = 256
# re_size = 224
# normalize = get_normalize()
# tra_train = transforms.Compose([
#     transforms.Resize(size),
#     transforms.ColorJitter( brightness=1 \
#                             contrast=1, \
#                             saturation=1, \
#                             hue=0.05),
#     transforms.ToTensor()
# ])

# tra_test = transforms.Compose([
#         transforms.Resize(size),
#         transforms.CenterCrop(re_size),
#         transforms.ToTensor(),
#         normalize])

# tra_mask= transforms.Compose([
#             transforms.Resize(size),
#             transforms.ToTensor()])

#######################################################################

# aug_prob = 0.5
# size = 256
# re_size = 224
# normalize = get_normalize()

# tra_fundus_1 = FundusAug.Compose([
#     FundusAug.Sharpness(prob = aug_prob),
#     FundusAug.Halo(size, prob=aug_prob),
#     FundusAug.Hole(size, prob=aug_prob),
#     FundusAug.Spot(size, prob=aug_prob),
#     FundusAug.Blur(prob=aug_prob)
# ])

# tra_fundus_2 = T.Compose([
#             T.RandomCrop(re_size),
#             T.RandomHorizontalFlip(),
#             T.RandomVerticalFlip(),
#             normalize])

#####################################################################

def func_transform(examples):

    # data = Image.open(self.data[index]).convert("RGB")
    images = [Image.open(path).convert("RGB") for path in examples['pr_image_path']]
    masks = [Image.open(path).convert("L") for path in examples['mask_path']]

    aug_images = [tra_train(image) for image in images]
    aug_mask = [tra_mask(mask) for mask in masks]
    
    examples["pixel_values"] = aug_images
    
    del examples["image_path"]
    del examples["pr_image_path"]
    del examples["mask_path"]
    
    return examples

def func_transform_test(examples):

    images = [Image.open(path).convert("RGB") for path in examples['pr_image_path']]

    aug_images = [tra_test(image) for image in images]
    
    examples["pixel_values"] = aug_images
    del examples["image_path"]
    del examples["pr_image_path"]
    del examples["mask_path"]
    return examples
    
train_ds = Dataset.from_pandas(train_dataset, preserve_index=False)
test_ds = Dataset.from_pandas(test_dataset, preserve_index=False)

prepared_ds_train = train_ds.with_transform(func_transform)
prepared_ds_test = test_ds.with_transform(func_transform_test)

prepared_ds_train = prepared_ds_train.shuffle(seed=42)
prepared_ds_test = prepared_ds_test.shuffle(seed=42)


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


In [10]:
print("rows in train_dataset: ", len(prepared_ds_train))
print("rows in test_dataset: ", len(prepared_ds_test))

# labels = prepared_ds_train.features["label"].names()
labels = [0, 1, 2, 3, 4]
label2id, id2label = dict(), dict()

for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label
    
print("ID2label: ", id2label)

rows in train_dataset:  57600
rows in test_dataset:  7026
ID2label:  {0: 0, 1: 1, 2: 2, 3: 3, 4: 4}


In [11]:
def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['label'] for x in batch])
    }

In [12]:
from sklearn.metrics import cohen_kappa_score, confusion_matrix
from sklearn.metrics import f1_score #, kappa
# from sklearn import metrics

import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions_proba, labels = eval_pred

    # print(predictions)
    predictions = np.argmax(predictions_proba, axis=1)
    # print(predictions)
    # print(labels)
    result_accuracy = accuracy.compute(predictions=predictions, references=labels)

    result = {
             'accuracy': np.mean([result_accuracy['accuracy']]),
             'kappa': np.mean([cohen_kappa_score(labels, predictions, weights = "quadratic")]),
             # 'quadratic_kappa': np.mean([kappa(labels, predictions, weights = "quadratic")]),
             'f1': np.mean([f1_score(labels, predictions, average='weighted')]),
             # 'roc_auc': np.mean([roc_auc_score(labels, predictions_proba, multi_class='ovr')])
             }

    cm = confusion_matrix(labels, predictions)
    print(cm)

    # print(cohen_kappa_score(labels, predictions))
    # print(result)

    return result


In [13]:
# from transformers import Swinv2ForImageClassification
from transformers import SwinForImageClassification

model = SwinForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True,
)

Some weights of SwinForImageClassification were not initialized from the model checkpoint at microsoft/swin-base-patch4-window12-384 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 1024]) in the checkpoint and torch.Size([5, 1024]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([5]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./swinv2-base",
    evaluation_strategy="steps",
    logging_steps=20,

    save_steps=20,
    eval_steps=20,
    save_total_limit=2,
    
    report_to="wandb",  # enable logging to W&B
    run_name="swin384_basicaug_67perdataset",  # name of the W&B run (optional)
    
    remove_unused_columns=False,

    learning_rate=2e-5,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    warmup_ratio=0.1,
    
    metric_for_best_model="kappa", 
    greater_is_better = True,
    load_best_model_at_end=True,
    
    push_to_hub=False
)

In [15]:
sample_ids = np.random.choice(len(prepared_ds_test), size=300, replace=False)
sample_ids_train = np.random.choice(len(prepared_ds_train), size=len(prepared_ds_train), replace=False)
val_ds = prepared_ds_test.select(sample_ids)
train_ds_shiffled = prepared_ds_train.select(sample_ids_train)

In [16]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds_train,
    # eval_dataset=prepared_ds_test,
    eval_dataset=val_ds,
    tokenizer=image_processor,
)

In [17]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

wandb: Currently logged in as: kalexu97 (alexu97skoltech). Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Accuracy,Kappa,F1
20,1.664800,1.789667,0.053333,-0.014118,0.081341
40,1.626800,1.656092,0.146667,-0.001004,0.212678
60,1.590000,1.518926,0.373333,0.127131,0.442337
80,1.581900,1.559388,0.230000,0.112102,0.293039
100,1.534300,1.400643,0.526667,0.224206,0.556899
120,1.483100,1.287525,0.600000,0.316210,0.605212
140,1.409400,1.336107,0.286667,0.228349,0.352430
160,1.330200,1.183840,0.590000,0.316924,0.598462
180,1.276900,1.368243,0.126667,0.226829,0.091753
200,1.276500,1.128807,0.363333,0.373969,0.431931


[[ 12   6   1  10 189]
 [  3   1   0   0  23]
 [  1   1   0   3  39]
 [  1   2   0   1   4]
 [  1   0   0   0   2]]
[[ 35  34   1  22 126]
 [  3   7   0   2  15]
 [  2  10   0   6  26]
 [  1   2   0   1   4]
 [  1   1   0   0   1]]
[[103  58   2  36  19]
 [ 14   7   0   3   3]
 [ 11  12   0  15   6]
 [  2   2   0   2   2]
 [  1   1   0   1   0]]
[[53 90  2 64  9]
 [ 8 11  0  7  1]
 [ 4 13  1 25  1]
 [ 0  2  1  4  1]
 [ 1  1  0  1  0]]
[[148  43   0   4  23]
 [ 17   7   0   2   1]
 [ 16  19   0   2   7]
 [  0   2   0   2   4]
 [  1   1   0   0   1]]
[[169  30   3   2  14]
 [ 18   4   2   2   1]
 [ 23  10   3   5   3]
 [  1   1   0   2   4]
 [  1   0   0   0   2]]
[[ 63 127   8   1  19]
 [  7  15   3   1   1]
 [  7  26   4   2   5]
 [  0   2   0   2   4]
 [  0   0   1   0   2]]
[[169  17   0  14  18]
 [ 17   4   0   5   1]
 [ 16   7   0  16   5]
 [  1   1   0   2   4]
 [  1   0   0   0   2]]
[[  8 188  10   4   8]
 [  1  23   2   1   0]
 [  0  31   4   8   1]
 [  0   2   0   2   4]
 [  0

wandb: Network error (ConnectTimeout), entering retry loop.


[[146  61   7   0   4]
 [ 11  13   3   0   0]
 [  3  12  23   5   1]
 [  0   1   2   4   1]
 [  0   1   0   0   2]]
[[183  30   3   0   2]
 [ 18   8   1   0   0]
 [ 10  11  18   5   0]
 [  1   1   1   4   1]
 [  1   0   0   0   2]]
[[ 71 138   6   0   3]
 [  3  21   3   0   0]
 [  0  20  16   8   0]
 [  1   1   0   5   1]
 [  0   1   0   0   2]]
[[ 15 168  29   2   4]
 [  1  19   6   1   0]
 [  0  12  19   9   4]
 [  0   1   1   5   1]
 [  0   0   1   0   2]]
[[183  24   8   0   3]
 [ 18   5   4   0   0]
 [  6   9  20   9   0]
 [  1   1   0   5   1]
 [  0   0   1   0   2]]
[[139  68   8   0   3]
 [ 11  12   4   0   0]
 [  4  13  20   7   0]
 [  1   1   0   5   1]
 [  0   1   0   0   2]]
[[141  62  12   0   3]
 [ 12  11   4   0   0]
 [  3  12  20   6   3]
 [  0   1   2   4   1]
 [  0   0   1   0   2]]
[[161  48   6   0   3]
 [ 13   9   5   0   0]
 [  3  10  23   5   3]
 [  1   1   1   4   1]
 [  0   0   1   0   2]]
[[121  89   6   0   2]
 [  9  15   3   0   0]
 [  2  17  22   2   1]
 [ 

In [18]:
metrics = trainer.evaluate(prepared_ds_test)
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

[[4481  494  184    4   18]
 [ 345  116   41    1    1]
 [ 299  144  467  101   23]
 [  10    5   54   98    8]
 [   4    1   29   17   81]]
***** eval metrics *****
  epoch                   =        2.0
  eval_accuracy           =     0.7462
  eval_f1                 =     0.7534
  eval_kappa              =     0.6949
  eval_loss               =     0.7531
  eval_runtime            = 0:02:14.96
  eval_samples_per_second =     52.056
  eval_steps_per_second   =      6.513
